<a href="https://colab.research.google.com/github/10udCryp7/TV-command-synthesis/blob/main/notebooks/Speaker_DataSplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Prep

In [ ]:
# aug speech
!gdown 1s0iFSfi-vnadzKuYz6mHIpXrdJw9YDWa

Downloading...
From (original): https://drive.google.com/uc?id=1s0iFSfi-vnadzKuYz6mHIpXrdJw9YDWa
From (redirected): https://drive.google.com/uc?id=1s0iFSfi-vnadzKuYz6mHIpXrdJw9YDWa&confirm=t&uuid=da2fd351-07a5-4a0e-81cc-e98c8f185757
To: /content/aug_speech.zip
100% 407M/407M [00:05<00:00, 80.2MB/s]


In [ ]:
!unzip -q aug_speech.zip

In [ ]:
import json

list_file = []
for folder in os.listdir('concat_speech/augmented'):
    for file in os.listdir(os.path.join('concat_speech/augmented', folder)):
        if file.endswith('.json'):
            with open(os.path.join('concat_speech/augmented', folder, file), 'r') as f:
                data = json.load(f)
                file_id = folder
                list_file.append(
                    {
                        'file_id': file_id,
                    }
                )


In [ ]:
df_file = pd.DataFrame(list_file)

# Get Speaker

In [ ]:
!gdown 10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk

Downloading...
From (original): https://drive.google.com/uc?id=10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk
From (redirected): https://drive.google.com/uc?id=10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk&confirm=t&uuid=9326f25d-3ca5-43db-933c-af5d1f1ae388
To: /content/synthesis.zip
100% 955M/955M [00:13<00:00, 68.9MB/s]


In [ ]:
!unzip -q synthesis.zip

In [ ]:
import os

for file in os.listdir('synthesis'):
    !unzip -q {os.path.join('synthesis', file)} -d {os.path.join('synthesis_all')}

In [ ]:
import json
import pandas as pd
speaker_list = []

for folder in os.listdir('synthesis_all'):
    for file in os.listdir(os.path.join('synthesis_all', folder)):
        if file.endswith('.json'):
            with open(os.path.join('synthesis_all', folder, file), 'r') as f:
                data = json.load(f)
                speech_id = data['id']
                speaker_id = data['ref_id']
                type_text = data['type']
                speaker_list.append(
                    {
                        'speech_id': speech_id,
                        'speaker_id': speaker_id,
                        'type': type_text
                    }
                )

In [ ]:
df_speaker = pd.DataFrame(speaker_list)

In [ ]:
df_speaker.to_csv('speaker_statistic_vctk.csv', index=False)

In [ ]:
df_speaker['file_id'] = df_speaker['type'] + '_' + df_speaker['speech_id']

# Merge

In [ ]:
df_merge = df_file.merge(df_speaker, on='file_id', how='left')

In [ ]:
df_merge

,file_id,speech_id,speaker_id,type
0,single_mix_06aac7c3,06aac7c3,p261_152,single_mix
1,single_mix_bd1cea4a,bd1cea4a,p269_115,single_mix
2,single_active_b2012b32,b2012b32,p288_262,single_active
3,non_active_971a2f77,971a2f77,p311_170,non_active
4,chain_active_9071fafe,9071fafe,p245_079,chain_active
...,...,...,...,...
3016,chain_active_98875b1c,98875b1c,p341_381,chain_active
3017,single_active_ffedb64a,ffedb64a,p228_216,single_active
3018,single_active_3dc7bc18,3dc7bc18,p247_010,single_active
3019,single_active_a2c0b27b,a2c0b27b,p351_264,single_active


In [ ]:
!pip install skmultilearn

ERROR: Could not find a version that satisfies the requirement skmultilearn (from versions: none)
ERROR: No matching distribution found for skmultilearn


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import jensenshannon  # dùng JS-divergence cho dễ

def best_speaker_split(df, test_size=0.2, n_trials=100, random_state=42):
    speaker_ids = df["speaker_id"].unique()
    best_score = float("inf")
    best_split = None

    for seed in range(random_state, random_state + n_trials):
        train_speakers, test_speakers = train_test_split(
            speaker_ids, test_size=test_size, random_state=seed
        )
        train_df = df[df["speaker_id"].isin(train_speakers)]
        test_df  = df[df["speaker_id"].isin(test_speakers)]

        # phân bố type
        p_train = train_df["type"].value_counts(normalize=True).sort_index()
        p_test  = test_df["type"].value_counts(normalize=True).sort_index()

        # align index
        all_types = sorted(set(df["type"]))
        p_train = p_train.reindex(all_types, fill_value=0)
        p_test  = p_test.reindex(all_types, fill_value=0)

        # JS divergence (càng nhỏ càng giống nhau)
        score = jensenshannon(p_train, p_test)

        if score < best_score:
            best_score = score
            best_split = (train_speakers, test_speakers, p_train, p_test)

    return best_split, best_score

# --- dùng thử ---
best_split, score = best_speaker_split(df_merge, test_size=0.2, n_trials=200)

train_speakers, test_speakers, p_train, p_test = best_split

train_df = df_merge[df_merge["speaker_id"].isin(train_speakers)].reset_index(drop=True)
test_df  = df_merge[df_merge["speaker_id"].isin(test_speakers)].reset_index(drop=True)

print("Best JS divergence:", score)
print("Train distribution:\n", p_train)
print("Test distribution:\n", p_test)


Best JS divergence: 0.008465693838139225
Train distribution:
 type
chain_active     0.222637
chain_mix        0.193615
non_active       0.232587
single_active    0.177446
single_mix       0.173715
Name: proportion, dtype: float64
Test distribution:
 type
chain_active     0.228243
chain_mix        0.193760
non_active       0.224959
single_active    0.182266
single_mix       0.170772
Name: proportion, dtype: float64


In [ ]:
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

In [ ]:
set_train = set(train_df['speaker_id'].unique())

In [ ]:
set_test = set(test_df['speaker_id'].unique())

In [ ]:
# overlap checking

set_train & set_test

set()

In [ ]:
train_df['type'].value_counts()

,count
type,
non_active,561
chain_active,537
chain_mix,467
single_active,428
single_mix,419


In [ ]:
test_df['type'].value_counts()

,count
type,
chain_active,139
non_active,137
chain_mix,118
single_active,111
single_mix,104


In [ ]:
import os
import shutil

def split_folders(base_dir, output_dir, train_df, test_df):
    os.makedirs(os.path.join(output_dir, "train", "aug"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "test", "aug"), exist_ok=True)

    # train
    for fid in train_df["file_id"]:
        src = os.path.join(base_dir, fid)
        dst = os.path.join(output_dir, "train", "aug", fid)
        if os.path.exists(src):
            shutil.copytree(src, dst)

    # test
    for fid in test_df["file_id"]:
        src = os.path.join(base_dir, fid)
        dst = os.path.join(output_dir, "test", "aug", fid)
        if os.path.exists(src):
            shutil.copytree(src, dst)

# --- dùng thử ---
base_dir = "concat_speech/augmented"           # thư mục gốc chứa file_id
output_dir = "dataset_split"   # nơi lưu train/test

split_folders(base_dir, output_dir, train_df, test_df)


In [ ]:
!zip vctk_split.zip dataset_split -r

Streaming output truncated to the last 5000 lines.
  adding: dataset_split/train/aug/chain_mix_03e589f3/ (stored 0%)
  adding: dataset_split/train/aug/chain_mix_03e589f3/chain_mix_03e589f3_aug.json (deflated 55%)
  adding: dataset_split/train/aug/chain_mix_03e589f3/chain_mix_03e589f3_aug.wav (deflated 16%)
  adding: dataset_split/train/aug/chain_active_2d5919d2/ (stored 0%)
  adding: dataset_split/train/aug/chain_active_2d5919d2/chain_active_2d5919d2_aug.json (deflated 48%)
  adding: dataset_split/train/aug/chain_active_2d5919d2/chain_active_2d5919d2_aug.wav (deflated 15%)
  adding: dataset_split/train/aug/chain_mix_b97ed886/ (stored 0%)
  adding: dataset_split/train/aug/chain_mix_b97ed886/chain_mix_b97ed886_aug.wav (deflated 23%)
  adding: dataset_split/train/aug/chain_mix_b97ed886/chain_mix_b97ed886_aug.json (deflated 56%)
  adding: dataset_split/train/aug/chain_active_3c9d98f7/ (stored 0%)
  adding: dataset_split/train/aug/chain_active_3c9d98f7/chain_active_3c9d98f7_aug.wav (deflate